In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1584491930225_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)

In [3]:
glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Only for sample data
enriched = glueContext.create_dynamic_frame.from_catalog(
    database="test", 
    table_name="enriched_events", 
    transformation_ctx = "datasource", 
    push_down_predicate = "loaded_date == '2020-03-16' and loaded_hour == '07'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print("Count: " + str(enriched.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Count: 987

In [6]:
enriched_df = enriched.toDF().dropDuplicates(['event_id', 'event_fingerprint'])
enriched_df.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[app_id: string, platform: string, etl_tstamp: string, collector_tstamp: string, dvce_tstamp: string, event: string, event_id: string, name_tracker: string, v_tracker: string, v_collector: string, v_etl: string, user_id: string, user_ipaddress: string, user_fingerprint: string, domain_userid: string, network_userid: string, geo_country: string, geo_region: string, geo_city: string, geo_zipcode: string, geo_latitude: string, geo_longitude: string, geo_region_name: string, ip_isp: string, ip_organization: string, ip_domain: string, ip_netspeed: string, page_url: string, page_title: string, page_referrer: string, page_urlscheme: string, page_urlhost: string, page_urlpath: string, page_urlquery: string, page_urlfragment: string, refr_urlscheme: string, refr_urlhost: string, refr_urlpath: string, refr_urlquery: string, refr_urlfragment: string, refr_medium: string, refr_source: string, refr_term: string, mkt_medium: string, mkt_source: string, mkt_term: string, mkt_content: string,

In [7]:
# enriched_df.select('event_vendor').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|        event_vendor|
+--------------------+
|com.snowplowanaly...|
|com.google.analyt...|
+--------------------+

In [7]:
#
# processing ga data
#
ga_df = enriched_df.filter(f.col('event_vendor') == 'com.google.analytics.measurement-protocol')
ga_df.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[app_id: string, platform: string, etl_tstamp: string, collector_tstamp: string, dvce_tstamp: string, event: string, event_id: string, name_tracker: string, v_tracker: string, v_collector: string, v_etl: string, user_id: string, user_ipaddress: string, user_fingerprint: string, domain_userid: string, network_userid: string, geo_country: string, geo_region: string, geo_city: string, geo_zipcode: string, geo_latitude: string, geo_longitude: string, geo_region_name: string, ip_isp: string, ip_organization: string, ip_domain: string, ip_netspeed: string, page_url: string, page_title: string, page_referrer: string, page_urlscheme: string, page_urlhost: string, page_urlpath: string, page_urlquery: string, page_urlfragment: string, refr_urlscheme: string, refr_urlhost: string, refr_urlpath: string, refr_urlquery: string, refr_urlfragment: string, refr_medium: string, refr_source: string, refr_term: string, mkt_medium: string, mkt_source: string, mkt_term: string, mkt_content: string,

In [186]:
_schema = StructType([
    StructField('documentLocationUrl', StringType(), True),
    StructField('documentHostName', StringType(), True),
    StructField('documentPath', StringType(), True),
    StructField('documentTitle', StringType(), True),
    StructField('category', StringType(), True),
    StructField('action', StringType(), True),
    StructField('label', StringType(), True),
    StructField('value', StringType(), True),
    StructField('userTimingCategory', StringType(), True),
    StructField('userTimingVariableName', StringType(), True),
    StructField('userTimingTime', StringType(), True),
    StructField('userTimingLabel', StringType(), True),
    StructField('pageLoadTime', StringType(), True),
    StructField('dnsTime', StringType(), True),
    StructField('pageDownloadTime', StringType(), True),
    StructField('redirectResponseTime', StringType(), True),
    StructField('tcpConnectTime', StringType(), True),
    StructField('serverResponseTime', StringType(), True),
    StructField('domInteractiveTime', StringType(), True),
    StructField('contentLoadTime', StringType(), True),])

schema_ue = StructType([
    StructField('schema', StringType(), True),
    StructField('data', StructType([
        StructField('schema', StringType(), True),
        StructField('data', _schema)
    ]))
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [187]:
active_columns = ['event_id', 'event_fingerprint', 'etl_tstamp', 'collector_tstamp', 'derived_tstamp', 'loaded_date', 'loaded_hour', 'v_tracker', 'v_collector', 'v_etl', 'user_ipaddress', 'network_userid', 'page_url', 'page_title', 'page_referrer', 'page_urlscheme', 'page_urlhost', 'page_urlport', 'page_urlpath', 'page_urlquery', 'page_urlfragment', 'refr_urlscheme', 'refr_urlhost', 'refr_urlport', 'refr_urlpath', 'refr_urlquery', 'refr_urlfragment', 'refr_medium', 'refr_source', 'refr_term', 'mkt_medium', 'mkt_source', 'mkt_term', 'mkt_content', 'mkt_campaign', 'useragent', 'br_lang', 'br_features_java', 'br_colordepth', 'doc_charset']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [188]:
# unstruct event data
ue_df = ga_df.withColumn('ue_json', f.from_json(f.col('unstruct_event'), schema_ue)).select(active_columns + ['unstruct_event', 'ue_json'])
ue_df = ue_df.select(active_columns + ['ue_json.data.schema'] + ['ue_json.data.data.'+column.name for column in _schema])
ue_df = ue_df.withColumn('event_type_ga', f.split(f.col('schema'),'/').getItem(1))

ue_df = ue_df.withColumnRenamed('documentLocationUrl', 'document_location_url')
ue_df = ue_df.withColumnRenamed('documentHostName', 'document_host_name')
ue_df = ue_df.withColumnRenamed('documentPath', 'document_path')
ue_df = ue_df.withColumnRenamed('documentTitle', 'document_title')
ue_df = ue_df.withColumnRenamed('category', 'category')
ue_df = ue_df.withColumnRenamed('action', 'action')
ue_df = ue_df.withColumnRenamed('label', 'label')
ue_df = ue_df.withColumnRenamed('value', 'value')
ue_df = ue_df.withColumnRenamed('userTimingCategory', 'user_timing_category')
ue_df = ue_df.withColumnRenamed('userTimingVariableName', 'user_timing_variable_name')
ue_df = ue_df.withColumnRenamed('userTimingTime', 'user_timing_time')
ue_df = ue_df.withColumnRenamed('userTimingLabel', 'user_timing_label')
ue_df = ue_df.withColumnRenamed('pageLoadTime', 'page_load_time')
ue_df = ue_df.withColumnRenamed('dnsTime', 'dns_time')
ue_df = ue_df.withColumnRenamed('pageDownloadTime', 'page_download_time')
ue_df = ue_df.withColumnRenamed('redirectResponseTime', 'redirect_response_time')
ue_df = ue_df.withColumnRenamed('tcpConnectTime', 'tcp_connect_time')
ue_df = ue_df.withColumnRenamed('serverResponseTime', 'server_response_time')
ue_df = ue_df.withColumnRenamed('domInteractiveTime', 'dom_interactive_time')
ue_df = ue_df.withColumnRenamed('contentLoadTime', 'content_load_time')

# ue_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
ue_df = ue_df.withColumn('event_date', f.to_date('derived_tstamp').cast(StringType()))
ue_df = ue_df.withColumn('event_hour', f.format_string("%02d", f.hour('derived_tstamp')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [191]:
schema_contexts = StructType(
    [
        StructField('schema', StringType(), True),
        StructField('data', ArrayType(StringType()), True)
    ]
)

schema_contexts_data = StructType(
    [
        StructField('schema', StringType(), True),
        StructField('data', StringType(), True)
    ]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [192]:
# Get contexts_df from events
contexts_df = ga_df.withColumn('contexts_json', f.from_json(f.col('contexts'), schema_contexts)).select('event_id', 'event_fingerprint', 'contexts', 'contexts_json')
contexts_df = contexts_df.select('event_id', 'event_fingerprint', f.explode('contexts_json.data').alias('datum')).withColumn('datum', f.from_json(f.col('datum'), schema_contexts_data)).select('event_id', 'event_fingerprint', 'datum.schema', 'datum.data')

contexts_df.cache()

# Split them based on its schema
contexts_df_by_schemas = {schema[0] : contexts_df.filter(f.col('schema') == schema[0]).drop('schema') for schema in contexts_df.select('schema').distinct().collect()}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [193]:
# UNDOCUMENTED: com.google.analytics/undocumented/jsonschema/1-0-0
#
#   a    : Random number used to link Google Analytics to Adsense
#   jid  : JoinID (binding the GA cookie to Double­Click cookie)
#   gjid : ??

schema_name = 'iglu:com.google.analytics/undocumented/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('a', StringType(), True),
        StructField('jid', StringType(), True),
        StructField('gjid', StringType(), True)])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('a', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('jid', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('gjid', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
# PRIVATE: com.google.analytics/private/jsonschema/1-0-0
#
# v   : SDK version number
# s   : Hit sequence - increments each time an event (inc pageview)
# u   : Verification code generated by GA analytics.js
# gid : analytics.js cookie used to distinguish users (24h lifetime)
# r   : seems to be linked to utmredir

schema_name = 'iglu:com.google.analytics/private/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('v', StringType(), True),
        StructField('s', StringType(), True),
        StructField('u', StringType(), True),
        StructField('gid', StringType(), True),
        StructField('r', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('v', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('s', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('u', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('gid', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('r', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
# GENERAL: com.google.analytics.measurement-protocol/general/jsonschema/1-0-0
#
# protocol_version : 
# tracking_id      : 
# anonymize_ip     : 
# data_source      : 
# queue_time       : 
# cache_buster     : 

schema_name = 'iglu:com.google.analytics.measurement-protocol/general/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('protocolVersion', StringType(), True),
        StructField('trackingId', StringType(), True),
        StructField('anonymizeIp', StringType(), True),
        StructField('dataSource', StringType(), True),
        StructField('queueTime', StringType(), True),
        StructField('cacheBuster', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('protocolVersion', 'protocol_version')
    _df = _df.withColumnRenamed('trackingId', 'tracking_id')
    _df = _df.withColumnRenamed('anonymizeIp', 'anonymize_ip')
    _df = _df.withColumnRenamed('dataSource', 'data_source')
    _df = _df.withColumnRenamed('queueTime', 'queue_time')
    _df = _df.withColumnRenamed('cacheBuster', 'cache_buster')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('protocol_version', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('tracking_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('anonymize_ip', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('data_source', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('queue_time', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('cache_buster', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [196]:
# USER: com.google.analytics.measurement-protocol/user/jsonschema/1-0-0
#
# client_id : 
# user_id   : 

schema_name = 'iglu:com.google.analytics.measurement-protocol/user/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('clientId', StringType(), True),
        StructField('userId', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('clientId', 'client_id')
    _df = _df.withColumnRenamed('userId', 'user_id')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('client_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('user_id', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [197]:
# SESSION: com.google.analytics.measurement-protocol/session/jsonschema/1-0-0
#
# session_control       : 
# ip_override           : 
# user_agent_override   :
# geographical_override :

schema_name = 'iglu:com.google.analytics.measurement-protocol/session/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('sessionControl', StringType(), True),
        StructField('ipOverride', StringType(), True),
        StructField('userAgentOverride', StringType(), True),
        StructField('geographicalOverride', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('sessionControl', 'session_control')
    _df = _df.withColumnRenamed('ipOverride', 'ip_override')
    _df = _df.withColumnRenamed('userAgentOverride', 'user_agent_override')
    _df = _df.withColumnRenamed('geographicalOverride', 'geographical_override')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('session_control', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('ip_override', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('user_agent_override', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('geographical_override', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [198]:
# TRAFFIC_SOURCE: com.google.analytics.measurement-protocol/traffic_source/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/traffic_source/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('documentReferrer', StringType(), True),
        StructField('campaignName', StringType(), True),
        StructField('campaignSource', StringType(), True),
        StructField('campaignMedium', StringType(), True),
        StructField('campaignKeyword', StringType(), True),
        StructField('campaignContent', StringType(), True),
        StructField('campaignId', StringType(), True),
        StructField('googleAdwordsId', StringType(), True),
        StructField('googleDisplayAdsId', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('documentReferrer', 'document_referrer')
    _df = _df.withColumnRenamed('campaignName', 'campaign_name')
    _df = _df.withColumnRenamed('campaignSource', 'campaign_source')
    _df = _df.withColumnRenamed('campaignMedium', 'campaign_medium')
    _df = _df.withColumnRenamed('campaignKeyword', 'campaign_keyword')
    _df = _df.withColumnRenamed('campaignContent', 'campaign_content')
    _df = _df.withColumnRenamed('campaignId', 'campaign_id')
    _df = _df.withColumnRenamed('googleAdwordsId', 'google_adwords_id')
    _df = _df.withColumnRenamed('googleDisplayAdsId', 'google_display_ads_id')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('document_referrer', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_name', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_source', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_medium', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_keyword', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_content', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('campaign_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('google_adwords_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('google_display_ads_id', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [199]:
# SYSTEM_INFO: com.google.analytics.measurement-protocol/system_info/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/system_info/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('screenResolution', StringType(), True),
        StructField('viewportSize', StringType(), True),
        StructField('documentEncoding', StringType(), True),
        StructField('screenColors', StringType(), True),
        StructField('userLanguage', StringType(), True),
        StructField('javaEnabled', StringType(), True),
        StructField('flashVersion', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('screenResolution', 'screen_resolution')
    _df = _df.withColumnRenamed('viewportSize', 'viewport_size')
    _df = _df.withColumnRenamed('documentEncoding', 'document_encoding')
    _df = _df.withColumnRenamed('screenColors', 'screen_colors')
    _df = _df.withColumnRenamed('userLanguage', 'user_language')
    _df = _df.withColumnRenamed('javaEnabled', 'java_enabled')
    _df = _df.withColumnRenamed('flashVersion', 'flash_version')

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('screen_resolution', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('viewport_size', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('document_encoding', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('screen_colors', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('user_language', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('java_enabled', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('flash_version', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [200]:
# HIT: com.google.analytics.measurement-protocol/hit/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/hit/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('type', StringType(), True),
        StructField('nonInteractionHit', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('type', 'hit_type')
    _df = _df.withColumnRenamed('nonInteractionHit', 'non_interaction_hit')

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('hit_type', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('non_interaction_hit', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
# PAGE_VIEW: com.google.analytics.measurement-protocol/page_view/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/page_view/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('documentLocationUrl', StringType(), True),
        StructField('documentHostName', StringType(), True),
        StructField('documentPath', StringType(), True),
        StructField('documentTitle', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('documentLocationUrl', '_document_location_url')
    _df = _df.withColumnRenamed('documentHostName', '_document_host_name')
    _df = _df.withColumnRenamed('documentPath', '_document_path')
    _df = _df.withColumnRenamed('documentTitle', '_document_title')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
    ue_df = ue_df.withColumn('document_location_url', f.coalesce(f.col('document_location_url'), f.col('_document_location_url')))
    ue_df = ue_df.withColumn('document_host_name', f.coalesce(f.col('document_host_name'), f.col('_document_host_name')))
    ue_df = ue_df.withColumn('document_path', f.coalesce(f.col('document_path'), f.col('_document_path')))
    ue_df = ue_df.withColumn('document_title', f.coalesce(f.col('document_title'), f.col('_document_title')))
    ue_df = ue_df.drop('_document_location_url', '_document_host_name', '_document_path', '_document_title')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [202]:
# APP: com.google.analytics.measurement-protocol/app/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/app/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('name', StringType(), True),
        StructField('id', StringType(), True),
        StructField('version', StringType(), True),
        StructField('installerId', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('name', 'app_name')
    _df = _df.withColumnRenamed('id', 'app_id')
    _df = _df.withColumnRenamed('version', 'app_version')
    _df = _df.withColumnRenamed('installerId', 'app_installer_id')

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('app_name', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('app_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('app_version', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('app_installer_id', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [203]:
# com.google.analytics.measurement-protocol/content_experiment/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/content_experiment/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('id', StringType(), True),
        StructField('variant', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('id', 'exp_id')
    _df = _df.withColumnRenamed('variant', 'exp_variant')

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('exp_id', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('exp_variant', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [204]:
# CUSTOM_DIMENSION: com.google.analytics.measurement-protocol/custom_dimension/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/custom_dimension/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('index', StringType(), True),
        StructField('value', StringType(), True),])
    
    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema]) \
             .withColumn('index', f.concat_ws('_', f.lit('custom_dimension'), f.col('index'))) \
             .groupby('event_id', 'event_fingerprint').pivot('index').agg(f.first('value'))

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
    
for i in range(1, 11):
    if f'custom_dimension_{i}' not in ue_df.columns:
        ue_df = ue_df.withColumn(f'custom_dimension_{i}', f.lit(None).cast(StringType()))




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
# CUSTOM_METRIC: com.google.analytics.measurement-protocol/custom_metric/jsonschema/1-0-0

schema_name = 'iglu:com.google.analytics.measurement-protocol/custom_metric/jsonschema/1-0-0'

if schema_name in contexts_df_by_schemas:
    _df = contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('index', StringType(), True),
        StructField('value', StringType(), True),])
    
    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema]) \
             .withColumn('index', f.concat_ws('_', f.lit('custom_metric'), f.col('index'))) \
             .groupby('event_id', 'event_fingerprint').pivot('index').agg(f.first('value'))

    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
    
for i in range(1, 11):
    if f'custom_metric_{i}' not in ue_df.columns:
        ue_df = ue_df.withColumn(f'custom_metric_{i}', f.lit(None).cast(StringType()))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
schema_derived_contexts = StructType(
    [
        StructField('schema', StringType(), True),
        StructField('data', ArrayType(StringType()), True)
    ]
)

schema_derived_contexts_data = StructType(
    [
        StructField('schema', StringType(), True),
        StructField('data', StringType(), True)
    ]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [208]:
# Get contexts_df from events
derived_contexts_df = ga_df.withColumn('derived_contexts_json', f.from_json(f.col('derived_contexts'), schema_derived_contexts)).select('event_id', 'event_fingerprint', 'derived_contexts', 'derived_contexts_json')
derived_contexts_df = derived_contexts_df.select('event_id', 'event_fingerprint', f.explode('derived_contexts_json.data').alias('datum')).withColumn('datum', f.from_json(f.col('datum'), schema_derived_contexts_data)).select('event_id', 'event_fingerprint', 'datum.schema', 'datum.data')

derived_contexts_df.cache()

# Split them based on its schema
derived_contexts_df_by_schemas = {schema[0] : derived_contexts_df.filter(f.col('schema') == schema[0]).drop('schema') for schema in derived_contexts_df.select('schema').distinct().collect()}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [209]:
# UA_PARSER_CONTEXT: com.google.analytics.measurement-protocol/hit/jsonschema/1-0-0

schema_name = 'iglu:com.snowplowanalytics.snowplow/ua_parser_context/jsonschema/1-0-0'

if schema_name in derived_contexts_df_by_schemas:
    _df = derived_contexts_df_by_schemas[schema_name]
    _schema = StructType([
        StructField('useragentFamily', StringType(), True),
        StructField('useragentMajor', StringType(), True),
        StructField('useragentMinor', StringType(), True),
        StructField('useragentPatch', StringType(), True),
        StructField('useragentVersion', StringType(), True),
        StructField('osFamily', StringType(), True),
        StructField('osMajor', StringType(), True),
        StructField('osMinor', StringType(), True),
        StructField('osPatch', StringType(), True),
        StructField('osPatchMinor', StringType(), True),
        StructField('osVersion', StringType(), True),
        StructField('deviceFamily', StringType(), True),])

    _df = _df.withColumn('data', f.from_json(f.col('data'), _schema)).select(['event_id', 'event_fingerprint'] + ['data.'+column.name for column in _schema])
    
    _df = _df.withColumnRenamed('useragentFamily', 'useragent_family')
    _df = _df.withColumnRenamed('useragentMajor', 'useragent_major')
    _df = _df.withColumnRenamed('useragentMinor', 'useragent_minor')
    _df = _df.withColumnRenamed('useragentPatch', 'useragent_patch')
    _df = _df.withColumnRenamed('useragentVersion', 'useragent_version')
    _df = _df.withColumnRenamed('osFamily', 'os_family')
    _df = _df.withColumnRenamed('osMajor', 'os_major')
    _df = _df.withColumnRenamed('osMinor', 'os_minor')
    _df = _df.withColumnRenamed('osPatch', 'os_patch')
    _df = _df.withColumnRenamed('osPatchMinor', 'os_patch_minor')
    _df = _df.withColumnRenamed('osVersion', 'os_version')
    _df = _df.withColumnRenamed('deviceFamily', 'device_family')
    
    ue_df = ue_df.join(_df, ['event_id', 'event_fingerprint'], how='left')
else:
    ue_df = ue_df.withColumn('useragent_family', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('useragent_major', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('useragent_minor', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('useragent_patch', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('useragent_version', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_family', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_major', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_minor', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_patch', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_patch_minor', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('os_version', f.lit(None).cast(StringType()))
    ue_df = ue_df.withColumn('device_family', f.lit(None).cast(StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [213]:
ue_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[event_id: string, event_fingerprint: string, etl_tstamp: string, collector_tstamp: string, derived_tstamp: string, loaded_date: string, loaded_hour: string, v_tracker: string, v_collector: string, v_etl: string, user_ipaddress: string, network_userid: string, page_url: string, page_title: string, page_referrer: string, page_urlscheme: string, page_urlhost: string, page_urlport: int, page_urlpath: string, page_urlquery: string, page_urlfragment: string, refr_urlscheme: string, refr_urlhost: string, refr_urlport: int, refr_urlpath: string, refr_urlquery: string, refr_urlfragment: string, refr_medium: string, refr_source: string, refr_term: string, mkt_medium: string, mkt_source: string, mkt_term: string, mkt_content: string, mkt_campaign: string, useragent: string, br_lang: string, br_features_java: string, br_colordepth: string, doc_charset: string, schema: string, document_location_url: string, document_host_name: string, document_path: string, document_title: string, categor

In [220]:
ue_df.write.parquet('s3://gx-data-staging-dev/snowplow/processed-test', partitionBy=['event_date', 'event_hour', 'loaded_date', 'loaded_hour'], mode='append')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…